# Customer Segmentation - Step 3: Subclustering at Department Level

In [9]:
# Input: cluster index
output_dataset = 'USER_SXM4690'
dept_temp_table = 'cs_temp_cluster'

# Output: department level clustering / subclustering indices
cs_output_subcluster_dept_table = 'cs_temp_subcluster_dept'
cs_output_table = 'cs_output_cluster_dept'
cs_output_table_copy = 'cs_par_pasa_output_cluster_dept'

In [2]:
import google.datalab.bigquery as bq
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.metrics import silhouette_score
from operator import itemgetter

def load_to_BQ_table(table_name, dataframe):
  
  dataframe.columns = dataframe.columns.str.replace('\s+', '_')
  dataframe.columns = dataframe.columns.str.replace('/', '_')
  dataframe.columns = dataframe.columns.str.replace('(', '_')
  dataframe.columns = dataframe.columns.str.replace(')', '_')
  dataframe = dataframe.where(pd.notnull(dataframe), None)
  
  import datalab.bigquery as bq
  bq_table = bq.Table(output_dataset + '.' + table_name)
  bq_table.delete()

  # create schema
  if not bq_table.exists():
    table_schema = bq.Schema.from_dataframe(dataframe)
    bq_table.create(schema = table_schema, overwrite = True)

  # load data
  bq_table.insert_data(dataframe)

  return(dataframe.head())


def cal_cluster_silhouette(dataset, clusters):
       
    clusterer = KMeans(init='k-means++', n_jobs=1, n_clusters=clusters, n_init=10, random_state=5)
    cluster_labels = clusterer.fit_predict(dataset)
    silhouette_avg = silhouette_score(dataset, cluster_labels, sample_size=min(10000, len(dataset.index)), random_state=5)
    
    print(str(clusters) + ': ' + str(silhouette_avg))
    return (clusters, silhouette_avg)
  
def auto_clustering(dataset, input_df_scaled, min_clusters, max_clusters, cluster_col):
  
  range_n_clusters = range(min_clusters, max_clusters+1)
  avg_silhouette = [] # create a list for number of clusters and average silhouette value

  for i in range_n_clusters:
    avg_silhouette.append(cal_cluster_silhouette(input_df_scaled, i))

  recommended_clusters = max(avg_silhouette,key=itemgetter(1))[0] # recommend num of clusters with highest silhouette

  mbk = KMeans(init='k-means++', n_jobs=1, n_clusters=recommended_clusters, n_init=10,  random_state=5)

  cluster_labels = mbk.fit_predict(input_df_scaled)

  dataset = dataset.assign(cluster_temp = cluster_labels + 1)

  # adjust cluster index by number of customers
  # reorder cluster number based on R12 sales
  cluster_size = dataset.groupby('cluster_temp').agg({'cust_id': 'count'}).reset_index()
  cluster_size['cluster_new'] = cluster_size['cust_id'].rank(ascending=False, method='first')

  # update new cluster index
  dataset = dataset.merge(cluster_size[['cluster_temp', 'cluster_new']], on='cluster_temp')
  dataset = dataset.drop(['cluster_temp'], axis=1).rename(columns = {'cluster_new': cluster_col})

  return (dataset)
  
def clustering(dataset, min_clusters, max_clusters, cluster_col):
  if cluster_col == 'cluster':
    input_df_scaled = dataset.iloc[:,4:len(dataset.columns)]
    input_df_scaled = StandardScaler().fit_transform(input_df_scaled)
    input_df_scaled = pd.DataFrame(input_df_scaled)
    
    dataset_output = auto_clustering(dataset, input_df_scaled, min_clusters, max_clusters, cluster_col)
    
  else:
    mbk = KMeans(init='k-means++', n_jobs=1, n_clusters=2, n_init=10,  random_state=5)
    
    if cluster_col == 'subcluster':
      clustering_sales = dataset[['tot_r12_sls_amt','tot_r12_txn_cnt']]
      clustering_sales = StandardScaler().fit_transform(clustering_sales)
      cluster_labels = mbk.fit_predict(clustering_sales)
        
    
    elif cluster_col == 'subcluster_class':
      clustering_sales = dataset.dept_r12_sls_amt
      cluster_labels = mbk.fit_predict(clustering_sales.reshape(-1,1))    
    
    dataset = dataset.assign(cluster_temp = cluster_labels + 1)

    # adjust cluster index by number of customers
    # reorder cluster number based on R12 sales
    cluster_size = dataset.groupby('cluster_temp').agg({'cust_id': 'count'}).reset_index()
    cluster_size['cluster_r1'] = cluster_size['cust_id'].rank(ascending=False, method='first')

    # update new cluster index
    dataset = dataset.merge(cluster_size[['cluster_temp', 'cluster_r1']], on='cluster_temp').drop(['cluster_temp'], axis=1)
  
    dataset_r1 = dataset[dataset.cluster_r1 == 2]
    
    dataset_r2 = dataset[dataset.cluster_r1 == 1].drop(['cluster_r1'], axis=1)
    
    if cluster_col == 'subcluster':    
      input_df_scaled = dataset_r2.iloc[:,4:len(dataset_r2.columns)]
    
    elif cluster_col == 'subcluster_class':      
      input_df_scaled= dataset_r2.iloc[:,1:(len(dataset_r2.columns)-1)]
    
    dataset_r2_output = auto_clustering(dataset_r2, input_df_scaled, min_clusters, max_clusters, cluster_col)
    
    dataset_r1_output = dataset_r1.assign(cluster_r1 = max(dataset_r2_output[cluster_col])+1).rename(columns = {'cluster_r1': cluster_col})

    dataset_output = dataset_r2_output.append(dataset_r1_output)
    
  return (dataset_output)


## Subclustering

In [3]:
# pull cluster index into dataframe
cs_cluster = bq.Table(output_dataset + '.' + dept_temp_table).to_dataframe()

In [4]:

# loop through cluster
cluster_list = cs_cluster['cluster'].drop_duplicates().tolist()
cluster_list.sort()

for i in cluster_list:
  print('Cluster ' + str(i))
  subcluster_data = cs_cluster[cs_cluster.cluster == i]
  subcluster_data = clustering(subcluster_data, 2, 10, 'subcluster')

  if i == min(cluster_list):
    cs_cluster_sub = subcluster_data

  else:
    cs_cluster_sub = cs_cluster_sub.append(subcluster_data, ignore_index=True)


Cluster 1.0
2: 0.17083537906165544
3: 0.15299033521361705
4: 0.12087068529721082
5: 0.12461767363489804
6: 0.12537209995943926
7: 0.12411600025260479
8: 0.11259154778968951
9: 0.10681805962805933
10: 0.10720167239468564
Cluster 2.0
2: 0.1423657379613573
3: 0.15966329168694973
4: 0.1421784984092078
5: 0.14754455734453636
6: 0.1555529767664545
7: 0.143566187127722
8: 0.142804110154844
9: 0.13806876826496908
10: 0.13111293325108458
Cluster 3.0
2: 0.4523538800984104
3: 0.3125155387676349
4: 0.27294222095854836
5: 0.2389242728593081
6: 0.22897218588496868
7: 0.23132341498309902
8: 0.21797059811353325
9: 0.20291024549038303
10: 0.20689192889590233
Cluster 4.0
2: 0.36594606696461357
3: 0.15830220298453182
4: 0.1568708682999916
5: 0.1333596052247254
6: 0.14731927142871898
7: 0.1448051878706251
8: 0.14920770561044244
9: 0.1436890267644811
10: 0.13690612519081335
Cluster 5.0
2: 0.41879768481941704
3: 0.2431865356403914
4: 0.18349360534153542
5: 0.15731391586274918
6: 0.16347907754722774
7: 0.156

In [5]:
# load result to BQ table
cs_cluster_output = cs_cluster.merge(cs_cluster_sub[['cust_id', 'subcluster']], on='cust_id')
load_to_BQ_table (table_name = cs_output_subcluster_dept_table, dataframe = cs_cluster_output)

,cust_id,s_trd_typ_desc,tot_r12_txn_cnt,tot_r12_sls_amt,D0021_LUMBER,D0022_BUILDING_MATERIALS,D0024_PAINT,D0030_MILLWORK,D023F_WALL_FLOOR_COVERING,D025H_HARDWARE__25H_,...,D026P_PLUMBING,D027E_ELECTRICAL,D027L_LIGHTING,D028I_GRD_INDOOR,D028O_GRD_OUTDOOR,D029A_APPLIANCES,D029B_KITCHEN_AND_BATH,D059S_STORAGE,cluster,subcluster
0,848266,Single Family Bldr,214.0,27573.0,0.030,0.078,0.088,0.001,0.180,0.097,...,0.026,0.044,0.186,0.042,0.042,0.021,0.027,0.067,1.0,1.0
1,689583,Remodeler,146.0,38838.0,0.054,0.181,0.038,0.197,0.034,0.024,...,0.085,0.059,0.060,0.001,0.002,0.003,0.209,0.015,1.0,1.0
2,2360859,Comm Prop Mgrs,204.0,50955.0,0.013,0.155,0.108,0.077,0.151,0.038,...,0.056,0.014,0.060,0.017,0.013,0.188,0.096,0.008,1.0,1.0
3,85905,Painter,1581.0,400526.0,0.149,0.119,0.111,0.081,0.073,0.053,...,0.113,0.090,0.049,0.015,0.011,0.010,0.072,0.012,1.0,1.0
4,667801,Commercial Constr.,30.0,38242.0,0.409,0.517,0.005,0.015,0.031,0.004,...,0.011,0.006,0.000,0.002,0.000,0.000,0.000,0.000,6.0,1.0


## Blend Filter Variables and Export

In [6]:
%%bq query -n cs_output
select f.*, d.* except (cust_id, s_trd_typ_desc), c.dept_nbr as dominate_dept, s.subcluster_class 
from `analytics-pricing-thd.USER_SXM4690.cs_filter` f
inner join `analytics-pricing-thd.USER_SXM4690.cs_temp_subcluster_dept` d
on cast(f.cust_id as int64) = cast(d.cust_id as int64)
left join `analytics-pricing-thd.USER_SXM4690.cs_temp_cluster_dom_dept` c
on cast(c.cluster as int64) = cast(d.cluster as int64)
left join (
  select distinct cust_id, subcluster_class 
  from `analytics-pricing-thd.USER_SXM4690.cs_output_subcluster_class`
) s
on cast(f.cust_id as int64) = cast(s.cust_id as int64)

In [7]:
cs_output_table = output_dataset + '.' + cs_output_table

In [8]:
%%bq execute -q cs_output -t $cs_output_table -m overwrite

cust_id,s_trd_typ_desc,ass_chan,pro_xtra_tenure,mgmt_tenure,dual_mgmt,pasa_algnmnt,par_algnmnt,pyrmd_tier,top_mkt_nm,top_mkt_rng,vly_rng,vpp_plus_bulk_rng,gm_rate_rng,tot_r12_txn_cnt,tot_r12_sls_amt,D0021_LUMBER,D0022_BUILDING_MATERIALS,D0024_PAINT,D0030_MILLWORK,D023F_WALL_FLOOR_COVERING,D025H_HARDWARE__25H_,D025T_TOOLS__25T_,D026P_PLUMBING,D027E_ELECTRICAL,D027L_LIGHTING,D028I_GRD_INDOOR,D028O_GRD_OUTDOOR,D029A_APPLIANCES,D029B_KITCHEN_AND_BATH,D059S_STORAGE,cluster,subcluster,dominate_dept,subcluster_class
4932181,Other,PASA,1-2 YEARS,NEVER MANAGED,N,LA,NOT PAR-MANAGED,B: > 35K,LA,A: > 90%,A: > 15%,D: >= 0%,B: > 30%,315.0,50478.0,0.002,0.326,0.098,0.025,0.164,0.086,0.088,0.031,0.049,0.019,0.026,0.011,0.022,0.023,0.03,1.0,2.0,,
3201334,Hospitality,PASA,2-3 YEARS,NEVER MANAGED,N,LA,NOT PAR-MANAGED,B: > 35K,LA,A: > 90%,E: <= -15%,D: >= 0%,A: > 45%,461.0,81542.0,0.117,0.086,0.073,0.041,0.063,0.083,0.063,0.069,0.065,0.107,0.027,0.02,0.098,0.061,0.027,1.0,1.0,,
501460,Remodeler,PASA,4-5 YEARS,NEVER MANAGED,N,LA,NOT PAR-MANAGED,B: > 35K,LA,A: > 90%,E: <= -15%,D: >= 0%,B: > 30%,306.0,46482.0,0.179,0.151,0.039,0.058,0.054,0.082,0.068,0.151,0.054,0.035,0.015,0.014,0.048,0.052,0.0,1.0,2.0,,
1642843,Painter,PASA,4-5 YEARS,NEVER MANAGED,N,LA,NOT PAR-MANAGED,B: > 35K,LA,B: > 80%,A: > 15%,D: >= 0%,A: > 45%,528.0,68088.0,0.055,0.106,0.113,0.048,0.061,0.068,0.1,0.103,0.282,0.017,0.019,0.0,0.013,0.014,0.001,1.0,1.0,,
496429,Property Manager,PAR,4-5 YEARS,3+ YEARS,Y,LA,"WESTERN, PACIFIC SOUTH",A: > 100K,LA,A: > 90%,D: BETWEEN -15% AND -5%,C: > 5%,C: > 15%,1164.0,767453.0,0.214,0.155,0.062,0.047,0.064,0.038,0.024,0.092,0.061,0.072,0.024,0.023,0.069,0.053,0.003,1.0,2.0,,
1901516,Remodeler,PASA,4-5 YEARS,NEVER MANAGED,N,LA,NOT PAR-MANAGED,B: > 35K,LA,A: > 90%,E: <= -15%,D: >= 0%,B: > 30%,661.0,57558.0,0.16,0.181,0.255,0.048,0.03,0.061,0.13,0.041,0.026,0.021,0.01,0.001,0.0,0.031,0.005,1.0,2.0,,
518395,Property Investor,PASA,4-5 YEARS,3+ YEARS,N,LA,NOT PAR-MANAGED,A: > 100K,LA,A: > 90%,A: > 15%,D: >= 0%,C: > 15%,583.0,226793.0,0.169,0.349,0.057,0.109,0.028,0.045,0.066,0.051,0.065,0.019,0.014,0.001,0.006,0.02,0.001,1.0,2.0,,
500013,Painter,PASA,4-5 YEARS,NEVER MANAGED,N,LA,NOT PAR-MANAGED,B: > 35K,LA,B: > 80%,B: BETWEEN 5% AND 15%,D: >= 0%,B: > 30%,274.0,67542.0,0.006,0.058,0.223,0.136,0.043,0.047,0.018,0.178,0.106,0.035,0.014,0.008,0.009,0.113,0.006,1.0,1.0,,
1694127,Hospitality,PASA,4-5 YEARS,NEVER MANAGED,N,LA,NOT PAR-MANAGED,B: > 35K,LA,A: > 90%,A: > 15%,D: >= 0%,B: > 30%,245.0,61046.0,0.286,0.289,0.096,0.011,0.07,0.059,0.089,0.039,0.018,0.004,0.026,0.01,0.0,0.001,0.003,1.0,2.0,,
841396,Spec. Restoration,PASA,4-5 YEARS,NEVER MANAGED,N,LA,NOT PAR-MANAGED,A: > 100K,LA,A: > 90%,A: > 15%,D: >= 0%,B: > 30%,940.0,139809.0,0.098,0.133,0.192,0.067,0.044,0.045,0.043,0.067,0.144,0.053,0.013,0.003,0.043,0.047,0.008,1.0,1.0,,


In [10]:
cs_output_table_copy = output_dataset + '.' + cs_output_table_copy

In [11]:
%%bq execute -q cs_output -t $cs_output_table_copy -m overwrite

cust_id,s_trd_typ_desc,ass_chan,pro_xtra_tenure,mgmt_tenure,dual_mgmt,pasa_algnmnt,par_algnmnt,pyrmd_tier,top_mkt_nm,top_mkt_rng,vly_rng,vpp_plus_bulk_rng,gm_rate_rng,tot_r12_txn_cnt,tot_r12_sls_amt,D0021_LUMBER,D0022_BUILDING_MATERIALS,D0024_PAINT,D0030_MILLWORK,D023F_WALL_FLOOR_COVERING,D025H_HARDWARE__25H_,D025T_TOOLS__25T_,D026P_PLUMBING,D027E_ELECTRICAL,D027L_LIGHTING,D028I_GRD_INDOOR,D028O_GRD_OUTDOOR,D029A_APPLIANCES,D029B_KITCHEN_AND_BATH,D059S_STORAGE,cluster,subcluster,dominate_dept,subcluster_class
665431,Remodeler,PASA,4-5 YEARS,NEVER MANAGED,N,LA,NOT PAR-MANAGED,B: > 35K,LA,B: > 80%,E: <= -15%,D: >= 0%,B: > 30%,317.0,49306.0,0.151,0.257,0.141,0.07,0.057,0.017,0.018,0.091,0.019,0.011,0.004,0.0,0.005,0.119,0.04,1.0,2.0,,
514023,Property Investor,PASA,4-5 YEARS,NEVER MANAGED,N,LA,NOT PAR-MANAGED,C: > 0K,LA,A: > 90%,E: <= -15%,C: > 5%,B: > 30%,262.0,28361.0,0.25,0.097,0.083,0.073,0.024,0.048,0.038,0.134,0.065,0.036,0.036,0.038,0.004,0.033,0.043,1.0,2.0,,
748089,Comm Prop Mgrs,PASA,4-5 YEARS,NEVER MANAGED,N,LA,NOT PAR-MANAGED,A: > 100K,LA,D: > 60%,D: BETWEEN -15% AND -5%,D: >= 0%,B: > 30%,606.0,119344.0,0.08,0.099,0.116,0.039,0.032,0.068,0.05,0.14,0.092,0.084,0.023,0.025,0.085,0.054,0.013,1.0,1.0,,
1627319,Remodeler,PASA,4-5 YEARS,NEVER MANAGED,N,LA,NOT PAR-MANAGED,B: > 35K,LA,A: > 90%,A: > 15%,D: >= 0%,B: > 30%,490.0,78273.0,0.093,0.109,0.03,0.123,0.073,0.062,0.109,0.143,0.07,0.083,0.015,0.006,0.003,0.075,0.008,1.0,1.0,,
662559,Commercial Constr.,PASA,4-5 YEARS,NEVER MANAGED,N,LA,NOT PAR-MANAGED,B: > 35K,LA,A: > 90%,B: BETWEEN 5% AND 15%,D: >= 0%,A: > 45%,487.0,77131.0,0.067,0.081,0.201,0.034,0.041,0.072,0.043,0.146,0.077,0.062,0.019,0.004,0.023,0.104,0.026,1.0,1.0,,
1622406,Commercial Constr.,PASA,4-5 YEARS,NEVER MANAGED,N,LA,NOT PAR-MANAGED,B: > 35K,LA,B: > 80%,A: > 15%,D: >= 0%,B: > 30%,261.0,54134.0,0.066,0.223,0.089,0.095,0.128,0.042,0.019,0.061,0.01,0.036,0.022,0.078,0.022,0.081,0.03,1.0,2.0,,
2128792,Property Manager,PASA,4-5 YEARS,NEVER MANAGED,N,LA,NOT PAR-MANAGED,B: > 35K,LA,B: > 80%,A: > 15%,D: >= 0%,B: > 30%,189.0,41831.0,0.071,0.052,0.036,0.04,0.149,0.051,0.066,0.097,0.069,0.042,0.062,0.009,0.137,0.092,0.027,1.0,1.0,,
3552464,Property Investor,PASA,2-3 YEARS,NEVER MANAGED,N,LA,NOT PAR-MANAGED,B: > 35K,LA,A: > 90%,A: > 15%,D: >= 0%,B: > 30%,309.0,90723.0,0.179,0.112,0.034,0.077,0.018,0.048,0.068,0.163,0.075,0.047,0.028,0.013,0.067,0.069,0.002,1.0,2.0,,
5215077,Remodeler,PASA,1-2 YEARS,NEVER MANAGED,N,LA,NOT PAR-MANAGED,B: > 35K,LA,A: > 90%,A: > 15%,D: >= 0%,B: > 30%,673.0,96874.0,0.114,0.101,0.1,0.058,0.091,0.041,0.048,0.092,0.046,0.046,0.033,0.008,0.037,0.17,0.017,1.0,1.0,,
1478748,Remodeler,PAR,4-5 YEARS,3+ YEARS,Y,LA,"WESTERN, PACIFIC SOUTH",A: > 100K,LA,B: > 80%,C: BETWEEN -5% AND 5%,C: > 5%,B: > 30%,1321.0,252674.0,0.069,0.106,0.156,0.075,0.072,0.062,0.034,0.113,0.049,0.093,0.011,0.003,0.011,0.142,0.005,1.0,1.0,,
